载入特征文件（）并建立网络，分类，进行训练，

In [11]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(0)

In [12]:
train_features = []
test_features = []
train_labels = []

with h5py.File("xception_bottleneck.h5", 'r') as h:
    train_features.append(np.array(h['train_features']))
    test_features.append(np.array(h['test_features']))
    train_labels = np.array(h['train_label'])

In [13]:
train_features = train_features[0]
test_features = test_features[0]

In [14]:
#验证数据集的形状是否正确
print("train_features shape:",np.shape(train_features))
print("test_features shape:",np.shape(test_features))
print("train_labels shape:",np.shape(train_labels))

train_features shape: (24998, 2048)
test_features shape: (12500, 2048)
train_labels shape: (24998,)


In [15]:
#是否打乱
train_features, train_labels = shuffle(train_features, train_labels)

构建网络

In [16]:
from keras.layers import Input, Dropout, Dense
from keras.models import Model
from keras import optimizers
from keras.callbacks import TensorBoard

In [17]:
inputlayer = Input(shape=(train_features.shape[1:]))

In [18]:
x = inputlayer

x = Dropout(0.85)(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.85)(x)
outlayer = Dense(1, activation='sigmoid')(x)

model = Model(input=inputlayer, output=outlayer)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [19]:
#优化器参考：https://keras.io/optimizers/
opt = optimizers.SGD(lr=0.0001, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(train_features,train_labels,nb_epoch=100,verbose=1,validation_split=0.2, batch_size=128, callbacks=[TensorBoard(log_dir='./log')])
model.save('model.h5')

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 19998 samples, validate on 5000 samples
Epoch 1/100
19998/19998 [==============================] - 1s 41us/step - loss: 0.9054 - acc: 0.5297 - val_loss: 0.5580 - val_acc: 0.8962
Epoch 2/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.7718 - acc: 0.5926 - val_loss: 0.4740 - val_acc: 0.9698
Epoch 3/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.6908 - acc: 0.6381 - val_loss: 0.4136 - val_acc: 0.9844
Epoch 4/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.6146 - acc: 0.6843 - val_loss: 0.3689 - val_acc: 0.9868
Epoch 5/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.5599 - acc: 0.7137 - val_loss: 0.3327 - val_acc: 0.9882
Epoch 6/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.5182 - acc: 0.7434 - val_loss: 0.3035 - val_acc: 0.9902
Epoch 7/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.4854 - acc: 0.7665 - val_loss: 0.2792 - v

19998/19998 [==============================] - 1s 39us/step - loss: 0.1339 - acc: 0.9547 - val_loss: 0.0485 - val_acc: 0.9922
Epoch 60/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.1314 - acc: 0.9557 - val_loss: 0.0480 - val_acc: 0.9922
Epoch 61/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.1326 - acc: 0.9552 - val_loss: 0.0475 - val_acc: 0.9922
Epoch 62/100
19998/19998 [==============================] - 1s 40us/step - loss: 0.1281 - acc: 0.9532 - val_loss: 0.0468 - val_acc: 0.9922
Epoch 63/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.1295 - acc: 0.9558 - val_loss: 0.0461 - val_acc: 0.9922
Epoch 64/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.1263 - acc: 0.9573 - val_loss: 0.0455 - val_acc: 0.9922
Epoch 65/100
19998/19998 [==============================] - 1s 39us/step - loss: 0.1268 - acc: 0.9560 - val_loss: 0.0451 - val_acc: 0.9922
Epoch 66/100
19998/19998 [==============

In [20]:
# 都预测为0.5的情况下，loss = 0.69314
pred = model.predict(train_features, batch_size=None, verbose=0, steps=None)
pred = pred.clip(min=0.005, max=0.995)

print(train_labels[:10])
print(pred.flatten()[:10])

[1 0 1 1 1 1 1 0 0 0]
[ 0.97093177  0.005       0.97584379  0.98585123  0.95861667  0.99206614
  0.995       0.005       0.005       0.005     ]
